# Prometheus PromQL Quickstart
이 노트북은 포트포워딩된 Prometheus 엔드포인트를 대상으로 PromQL을 실행하고 결과를 판다스로 확인하는 예제입니다.


In [1]:
import sys
import subprocess

def ensure_package(package):
    try:
        __import__(package)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

for pkg in ('requests', 'pandas'):
    ensure_package(pkg)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 9.5 MB/s eta 0:00:00


In [7]:
import os
import requests
import pandas as pd
from datetime import datetime

PROMETHEUS_URL = os.getenv('PROMETHEUS_URL', 'http://localhost:9090')
print(f'Using Prometheus endpoint: {PROMETHEUS_URL}')


Using Prometheus endpoint: http://localhost:9090


## Helper 함수
PromQL 쿼리를 실행하고 JSON 결과를 데이터프레임으로 변환합니다.

In [3]:
def run_promql(query: str, time: datetime | None = None):
    params = {'query': query}
    if time is not None:
        params['time'] = time.timestamp()
    response = requests.get(f'{PROMETHEUS_URL}/api/v1/query', params=params, timeout=10)
    response.raise_for_status()
    payload = response.json()
    if payload.get('status') != 'success':
        raise RuntimeError(f"Prometheus query failed: {payload}")
    return payload['data']['result']

def promql_to_dataframe(result):
    records = []
    for item in result:
        metric = item.get('metric', {})
        value = item.get('value')
        if value:
            records.append({**metric, 'value': float(value[1]), 'timestamp': datetime.fromtimestamp(value[0])})
    return pd.DataFrame(records)


## 예시 1: Node Exporter 상태 (up)
각 노드의 Node Exporter가 살아 있는지 확인합니다.

In [4]:
query = 'up{job="node-exporter"}'
result = run_promql(query)
df = promql_to_dataframe(result)
df[['instance', 'value', 'timestamp']]


,instance,value,timestamp
0,10.0.1.6:9100,1.0,2025-09-28 15:06:18.474
1,10.0.1.8:9100,1.0,2025-09-28 15:06:18.474
2,10.0.1.7:9100,1.0,2025-09-28 15:06:18.474


## 예시 2: CPU 사용률
최근 5분 평균 CPU idle 비율을 이용해서 사용률(%)을 계산합니다.

In [5]:
cpu_query = '100 - (avg by (instance)(rate(node_cpu_seconds_total{mode="idle", job="node-exporter"}[5m])) * 100)'
cpu_df = promql_to_dataframe(run_promql(cpu_query))
cpu_df[['instance', 'value']].rename(columns={'value': 'cpu_usage_percent'})


,instance,cpu_usage_percent
0,10.0.1.6:9100,0.840351
1,10.0.1.8:9100,0.459649
2,10.0.1.7:9100,0.414035


## 예시 3: 메모리 사용률
총 메모리 대비 사용 중인 메모리를 계산합니다.

In [8]:
mem_query = '(1 - node_memory_MemAvailable_bytes{job="node-exporter"} / node_memory_MemTotal_bytes{job="node-exporter"}) * 100'
mem_df = promql_to_dataframe(run_promql(mem_query))
mem_df[['instance', 'value']].rename(columns={'value': 'memory_usage_percent'})


,instance,memory_usage_percent
0,10.0.1.6:9100,62.312350
1,10.0.1.8:9100,46.787572
2,10.0.1.7:9100,46.070978
